## Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import math
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from typing import Dict, List, Optional, Tuple
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import os
from sklearn.decomposition import PCA

## Load data

In [ ]:
TRAIN = pd.read_csv('data/preprocessed_train.csv')
VAL = pd.read_csv('data/preprocessed_val.csv')
TEST = pd.read_csv('data/preprocessed_test.csv')

TRAIN.head()

In [ ]:
COLUMNS = list(TRAIN.columns[:-1])
COLUMNS[2] = "fam_hist"
COLUMNS

In [ ]:
y_MAPPING = {
    0: "Normal",
    1: "Obesity I",
    2: "Obesity II",
    3: "Obesity III",
    4: "Overweight I",
    5: "Overweight II",
    6: "Underweight"
}

Gender_MAPPPING = {
    0: "Female",
    1: "Male"
}

fam_hist_MAPPING = {
    0: "No",
    1: "Yes"
}

FAVC_MAPPING = {
    0: "No",
    1: "Yes"
}

CAEC_MAPPING = {
    0: "Always",
    1: "Frequently",
    2: "Sometimes",
    3: "No"
}

SMOKE_MAPPING = {
    0: "No",
    1: "Yes"
}

SCC_MAPPING = {
    0: "No",
    1: "Yes"
}

CALC_MAPPING = {
    0: "Frequently",
    1: "Sometimes",
    2: "No"
}

MTRANS_MAPPING = {
    0: "Automobile",
    1: "Bike",
    2: "Motorbike",
    3: "Public_Transportation",
    4: "Walking",
}

## Implementation

In [ ]:
class TwoGaussianMixturePrior:
    def __init__(
        self, 
        sigma_1: float = 1, 
        sigma_2: float = 1e-6, 
        mixing: float = 0.5,
    ):
        super().__init__()
        
        self.mixing = mixing
        
        self.w_prior_1 = torch.distributions.Normal(0, sigma_1)
        self.w_prior_2 = torch.distributions.Normal(0, sigma_2)
        
        self.b_prior_1 = torch.distributions.Normal(0, sigma_1)
        self.b_prior_2 = torch.distributions.Normal(0, sigma_2)
        
    def log_prob(self, weights: torch.Tensor, biases: torch.Tensor):
        w_log_prior_1 = self.w_prior_1.log_prob(weights).exp()
        w_log_prior_2 = self.w_prior_2.log_prob(weights).exp()
        
        w_prior = self.mixing * w_log_prior_1 + (1 - self.mixing) * w_log_prior_2
        
        b_log_prior_1 = self.b_prior_1.log_prob(biases).exp()
        b_log_prior_2 = self.b_prior_2.log_prob(biases).exp()
        
        b_prior = self.mixing * b_log_prior_1 + (1 - self.mixing) * b_log_prior_2
    
        return w_prior.log().mean() + b_prior.log().mean()

In [ ]:
class BayesianLinear(nn.Module):
    def __init__(
        self, 
        num_input_features: int,
        num_output_features: int,
        prior: TwoGaussianMixturePrior,
    ):
        """Implement initialization of weights and biases values"""
        super().__init__()
        
        self.prior = prior
        
        self.last_weights_ = None
        self.last_biases_ = None
        
        # Define weights parameters and initialize them using uniform distribution
        self.weights_mean = nn.Parameter(torch.empty(num_input_features, num_output_features))
        self.weights_std = nn.Parameter(torch.empty(num_input_features, num_output_features))
        nn.init.uniform_(self.weights_mean, a=-0.5, b=0.5)
        nn.init.uniform_(self.weights_std, a=-5, b=-3)
        
        # Define biases parameters and initialize them using uniform distribution
        self.biases_mean = nn.Parameter(torch.empty(num_output_features))
        self.biases_std = nn.Parameter(torch.empty(num_output_features))
        nn.init.uniform_(self.biases_mean)
        nn.init.uniform_(self.biases_std)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Implement forward inference using reparametrization trick"""
        
        weights = None
        biases = None
        
        eps_weights = torch.normal(torch.zeros(self.weights_mean.shape), torch.ones(self.weights_mean.shape))
        eps_biases = torch.normal(torch.zeros(self.biases_mean.shape), torch.ones(self.biases_mean.shape))

        weights = self.weights_mean + torch.log(1 + torch.exp(self.weights_std)) * eps_weights
        biases = self.biases_mean + torch.log(1 + torch.exp(self.biases_std)) * eps_biases
        
        self.last_weights_ = weights
        self.last_biases_ = biases

        return x @ weights + biases
        
    def prior_log_prob(self) -> torch.Tensor:
        """Calculates the prior log prob of sampled weights and biases."""
        return self.prior.log_prob(weights=self.last_weights_, biases=self.last_biases_)
        
    def variational_log_prob(self) -> torch.Tensor:
        """Implement the variational log prob."""
        weights_std = torch.log(1 + torch.exp(self.weights_std))
        biases_std = torch.log(1 + torch.exp(self.biases_std))

        w_posterior = torch.distributions.Normal(self.weights_mean, weights_std)
        b_posterior = torch.distributions.Normal(self.biases_mean, biases_std)
        
        return w_posterior.log_prob(self.last_weights_).mean() + b_posterior.log_prob(self.last_biases_).mean()

In [ ]:
class BayesianMLP(nn.Module):
    def __init__(
        self, 
        num_input_features: int,
        num_hidden_features: int,
        num_output_classes: int,
        prior: TwoGaussianMixturePrior,
    ):
        super().__init__()
        
        self.layer_1 = BayesianLinear(
            num_input_features, num_hidden_features, 
            prior=prior,
        )
        self.layer_2 = BayesianLinear(
            num_hidden_features, num_output_classes, 
            prior=prior,
        )
        
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x: torch.Tensor, use_softmax=True) -> torch.Tensor:
        x = self.sigmoid(self.layer_1(x))
        x = self.layer_2(x)
        if use_softmax:
            x = self.softmax(x)
        return x
        
    def prior_log_prob(self) -> torch.Tensor:
        log_prob = 0
        for module in self.modules():
            if isinstance(module, BayesianLinear):
                log_prob += module.prior_log_prob()
        return log_prob
        
    def variational_log_prob(self) -> torch.Tensor:
        log_prob = 0
        for module in self.modules():
            if isinstance(module, BayesianLinear):
                log_prob += module.variational_log_prob()
        return log_prob

In [ ]:
from typing import Union, Tuple

class ELBO(nn.Module):
    def __init__(self, N: int):
        super().__init__()
        
        self.N = N
        self.nll = nn.NLLLoss(reduction="none")
        
    def forward(
        self, 
        model: nn.Module, 
        inputs: torch.Tensor,
        targets: torch.Tensor,
        *,
        return_predictions: bool = False,
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        predictions = []
        log_posteriors = []
        log_priors = []
        
        for _ in range(self.N):
            preds = model(inputs)
            log_priors.append(model.prior_log_prob())
            log_posteriors.append(model.variational_log_prob())
            # NLLLoss input is log probabilities
            predictions.append(preds)
            
        loss = 0

        sum_log_posterior = sum(log_posteriors)
        sum_log_prior = sum(log_priors)

        sum_loss_nll = 0
        for pred in predictions:
            sum_loss_nll += torch.sum(self.nll(pred.log(), targets))
        
        # loss_nll is negative that's why we need to add it
        loss = (sum_log_posterior - sum_log_prior + sum_loss_nll) / self.N
        
        if return_predictions:
            return loss, torch.stack(predictions, dim=-1)
        return loss

## Utils

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data.values).float()
        self.labels = torch.tensor(labels).long()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
FLOAT_EPS = torch.finfo(torch.float32).eps

class Analyzer:
    def __init__(self, model: nn.Module, dataset: Dataset, num_samplings: int):
        self.model = model
        self.dataset = dataset
        self.num_samplings = num_samplings

        self.model.eval()

        self._preds: Optional[torch.Tensor] = None
        self._trues: Optional[torch.Tensor] = None
        self._data: Optional[torch.Tensor] = None

        self._retrieve_predictions()

    def reset_state(self):
        self._preds = None
        self._trues = None

    def _retrieve_predictions(self):
        loader = DataLoader(
            self.dataset, batch_size=32, shuffle=False, drop_last=False
        )

        trues = []
        preds = []
        data = []
        for inputs, targets in loader:
            data.append(inputs)
            trues.append(targets)
            if self.num_samplings <= 1:
                preds.append(self.model(inputs).detach())
            else:
                loc_preds = []
                for _ in range(self.num_samplings):
                    loc_preds.append(self.model(inputs).detach())
                preds.append(torch.stack(loc_preds, dim=-1))

        self._preds = torch.cat(preds, dim=0)
        self._trues = torch.cat(trues, dim=0)
        self._data = torch.cat(data, dim=0)

    @classmethod
    def _get_entropy(cls, matrix: torch.Tensor) -> torch.Tensor:
        return -(matrix * matrix.clamp_min(FLOAT_EPS).log()).sum(dim=-1)

    def get_top_k_high_confidence_mistakes(
        self, k: int = 10
    ) -> Tuple[torch.Tensor, ...]:
        
        if len(self._preds.shape) == 3:
            mean_preds = self._preds.mean(dim=-1)
        else:
            mean_preds = self._preds
            
        where_mistakes = mean_preds.argmax(dim=-1) != self._trues
        preds_with_mistakes = mean_preds[where_mistakes]
        data = self._data[where_mistakes]
        trues = self._trues[where_mistakes]

        top_high_confidence_indices = self._get_entropy(
            preds_with_mistakes
        ).argsort(descending=False)[:k]
        
        return (
            data[top_high_confidence_indices],
            self._preds[where_mistakes][top_high_confidence_indices],
            trues[top_high_confidence_indices],
        )

    def get_top_k_low_confidence_mistakes(
        self, k: int = 10
    ) -> Tuple[torch.Tensor, ...]:
        
        if len(self._preds.shape) == 3:
            mean_preds = self._preds.mean(dim=-1)
        else:
            mean_preds = self._preds
            
        where_mistakes = mean_preds.argmax(dim=-1) != self._trues
        preds_with_mistakes = mean_preds[where_mistakes]
        data = self._data[where_mistakes]
        trues = self._trues[where_mistakes]

        top_low_confidence_indices = self._get_entropy(
            preds_with_mistakes
        ).argsort(descending=True)[:k]
        
        return (
            data[top_low_confidence_indices],
            self._preds[where_mistakes][top_low_confidence_indices],
            trues[top_low_confidence_indices],
        )

    def get_top_k_high_confidence_correct(
        self, k: int = 10
    ) -> Tuple[torch.Tensor, ...]:
        
        if len(self._preds.shape) == 3:
            mean_preds = self._preds.mean(dim=-1)
        else:
            mean_preds = self._preds
            
        where_correct = mean_preds.argmax(dim=-1) == self._trues
        preds_correct = mean_preds[where_correct]
        data = self._data[where_correct]
        trues = self._trues[where_correct]

        top_high_confidence_indices = self._get_entropy(
            preds_correct
        ).argsort(descending=False)[:k]
        
        return (
            data[top_high_confidence_indices],
            self._preds[where_correct][top_high_confidence_indices],
            trues[top_high_confidence_indices],
        )

    def get_top_k_low_confidence_correct(
        self, k: int = 10
    ) -> Tuple[torch.Tensor, ...]:
        
        if len(self._preds.shape) == 3:
            mean_preds = self._preds.mean(dim=-1)
        else:
            mean_preds = self._preds
            
        where_correct = mean_preds.argmax(dim=-1) == self._trues
        preds_correct = mean_preds[where_correct]
        data = self._data[where_correct]
        trues = self._trues[where_correct]

        top_low_confidence_indices = self._get_entropy(
            preds_correct
        ).argsort(descending=True)[:k]
        
        return (
            data[top_low_confidence_indices],
            self._preds[where_correct][top_low_confidence_indices],
            trues[top_low_confidence_indices],
        )

In [ ]:
def visualize_weights(params: torch.Tensor, name: str) -> plt.Figure:
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    sns.kdeplot(params.view((-1,)).detach().numpy(), ax=ax)
    ax.set_title(f"Kernel density plot of {name}")
    return fig


def show_learning_curve(
    train_metrics: Dict[str, List[float]], test_metrics: Dict[str, List[float]]
) -> plt.Figure:
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.plot(train_metrics["step"], train_metrics["loss"], label="train")
    ax.plot(test_metrics["step"], test_metrics["loss"], label="test")
    ax.set_xlabel("Training step")
    ax.set_ylabel("Loss")
    ax.set_title("Learning curve")
    plt.legend()
    plt.show()


def show_accuracy_curve(
    train_metrics: Dict[str, List[float]], test_metrics: Dict[str, List[float]]
) -> plt.Figure:
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    plt.plot(train_metrics["step"], train_metrics["acc"], label="train")
    ax.plot(test_metrics["step"], test_metrics["acc"], label="test")
    ax.set_xlabel("Training step")
    ax.set_ylabel("Accuracy")
    ax.set_title("Accuracy curve")
    plt.legend()
    plt.show()

In [ ]:
def convert_tensor_to_text(info_tensor):
    text_to_display = ""
    for info, col_name in zip(info_tensor, COLUMNS):
        if col_name == "Gender":
            text = Gender_MAPPPING[int(info.item())]
        elif col_name == "fam_hist":
            text = fam_hist_MAPPING[int(info.item())]
        elif col_name == "FAVC":
            text = FAVC_MAPPING[int(info.item())]
        elif col_name == "CAEC":
            text = CAEC_MAPPING[int(info.item())]
        elif col_name == "SMOKE":
            text = SMOKE_MAPPING[int(info.item())]
        elif col_name == "SCC":
            text = SCC_MAPPING[int(info.item())]
        elif col_name == "CALC":
            text = CALC_MAPPING[int(info.item())]
        elif col_name == "MTRANS":
            text = MTRANS_MAPPING[int(info.item())]
        else:
            text = f'{info.item():.4f}'
        
        text_to_display = text_to_display + f'{col_name}:{text}\n'
        
    return text_to_display

def visualize_samples(
    data: torch.Tensor,
    preds: torch.Tensor,
    trues: torch.Tensor,
    num_cols: int = 10
) -> None:

    target_values = list(y_MAPPING.values())
    no_target_values = len(target_values)

    fig, axes = plt.subplots(
        1, num_cols, figsize=(3 * num_cols, 2 * 2)
    )

    is_with_multiple_samplings = len(preds.shape) == 3

    x = 0
    for info, pred, true in zip(
        data, preds, trues
    ):  # type: torch.Tensor, torch.Tensor, torch.Tensor
        if is_with_multiple_samplings:
            pred_mean = pred.mean(dim=-1)
            pred_std = pred.std(dim=-1)
        else:
            pred_mean = pred
            pred_std = 0

        pred_class = pred_mean.argmax().item()
        true_class = true.item()
        axes[x].set_title(f"Pred: {y_MAPPING[pred_class]} \n True: {y_MAPPING[true_class]}")
        axes[x].bar(np.arange(no_target_values), pred_mean, yerr=pred_std)
        axes[x].set_ylim(0, 1)

        if pred_class == true_class:
            axes[x].bar([pred_class], pred_mean[[pred_class]], color="green")
        else:
            axes[x].bar([true_class], pred_mean[[true_class]], color="yellow")
            axes[x].bar([pred_class], pred_mean[[pred_class]], color="red")

        axes[x].set_xticks(range(len(target_values)))
        axes[x].set_xticklabels(target_values)
        axes[x].tick_params(axis='x', labelrotation=90)

        
        axes[x].text(0, -1.2, convert_tensor_to_text(info))
        x += 1
            
    plt.show()

In [ ]:
def set_seed(seed=0):
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)

def fit_elbo(
    model: nn.Module,
    train_dataset: Dataset,
    val_dataset: Dataset,
    loss_fn: nn.Module,
    batch_size: int,
    epochs: int,
    optimizer: Optimizer,
) -> Tuple[Dict[str, List[float]], ...]:
    train_metrics = {"loss": [], "acc": [], "step": []}
    test_metrics = {"loss": [], "acc": [], "step": []}

    global_step = 0

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, drop_last=True
    )

    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, drop_last=False
    )

    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1} / {epochs}")

        # train step
        model.train()
        pbar = tqdm(train_loader)
        for inputs, targets in pbar:
            optimizer.zero_grad()

            loss, y_predictions = loss_fn(
                model, inputs, targets, return_predictions=True
            )
            loss.backward() 
            optimizer.step()
            
            y_predictions = y_predictions.mean(dim=-1)
            
            accuracy = (
                (y_predictions.argmax(dim=1) == targets)
                .float()
                .mean()
            )

            train_metrics["loss"].append(loss.item())
            train_metrics["acc"].append(accuracy.item())
            train_metrics["step"].append(global_step)
            global_step += 1
            pbar.update(1)
            
        pbar.close()

        # validating step
        model.eval()

        preds = []
        trues = []
        total_batches = 0
        total_loss = 0.0
        for inputs, targets in val_loader:
            loss, y_predictions = loss_fn(
                model, inputs, targets, return_predictions=True
            )
            
            y_predictions = y_predictions.mean(dim=-1)
            total_batches += 1

            total_loss += loss.item()

            trues.append(targets)
            preds.append(y_predictions)

        preds = torch.cat(preds, dim=0)
        trues = torch.cat(trues, dim=0)

        val_acc = (
            (preds.argmax(dim=1) == trues).float().mean().item()
        )

        test_metrics["loss"].append(total_loss / total_batches)
        test_metrics["acc"].append(val_acc)
        test_metrics["step"].append(global_step)

    return train_metrics, test_metrics


## Tests - hyperparameters search

In [ ]:
# UTILS
BATCH_SIZE = 32
NO_EPOCHS = 30
MODEL_NAME = "model.pt"
TRAIN_CSV_NAME = "train.csv"
VAL_CSV_NAME = "val.csv"

def get_results(folder):
    train_df = pd.read_csv(os.path.join(folder, TRAIN_CSV_NAME))
    val_df = pd.read_csv(os.path.join(folder, VAL_CSV_NAME))

    return (
        min(train_df["loss"]),
        max(train_df["acc"]), 
        min(val_df['loss']), 
        max(val_df['acc'])
    )

def get_model(folder):
    return torch.load(os.path.join(folder, MODEL_NAME))

def visualise_results(folder):
    train_metrics = pd.read_csv(os.path.join(folder, TRAIN_CSV_NAME))
    test_metrics = pd.read_csv(os.path.join(folder, VAL_CSV_NAME))
    
    show_learning_curve(train_metrics, test_metrics)
    show_accuracy_curve(train_metrics, test_metrics)

def calculate_metrics(model, X_test, y_test):
    with torch.no_grad():
        model_input = torch.tensor(X_test.values).float()
        y_pred = model(model_input)

    y_pred_argmax = torch.argmax(y_pred, dim=1)

    accuracy = accuracy_score(y_test, y_pred_argmax)
    prec = precision_score(y_test, y_pred_argmax, average='micro')
    rec = recall_score(y_test, y_pred_argmax, average='micro')
    f1 = f1_score(y_test, y_pred_argmax, average='micro')
    roc = roc_auc_score(y_test, y_pred, average='micro', multi_class='ovr')

    return {
        "Acccuracy": accuracy,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1,
        "ROC AUC Score": roc
    }

def plot_credible_intervals(X_test, y_test, class_index, samples=100):
    y_filter = y_test[y_test==class_index]
    X_filter = X_test.iloc[y_filter.index]

    X_tmp = torch.tensor(X_filter.values).float()
    y_samp = np.zeros((samples, X_filter.shape[0]))
    for s in range(samples):
        y_tmp = best_model(X_tmp, use_softmax=False).detach().numpy()
        y_samp[s] = y_tmp[:, class_index]

    pca = PCA(1)
    X_plot = pca.fit_transform(X_tmp).reshape(-1)
    
    X_plot = pd.Series(X_plot).sort_values()
    y_samp = y_samp[X_plot.index]

    plt.plot(X_plot, np.mean(y_samp, axis=0), label='Mean Posterior Predictive')
    plt.fill_between(X_plot, np.percentile(y_samp, 5, axis = 0), np.percentile(y_samp, 95, axis = 0), alpha = 0.25, label='90% Confidence')
    plt.title(f"Confidence interval for class {y_MAPPING[class_index]}")
    plt.xlabel("X after PCA transform")
    plt.ylabel("Prediction (without softmax)")
    plt.legend()
    plt.show()

In [ ]:
X_train = TRAIN.drop(['y'], axis=1)
y_train = TRAIN['y']
X_val = VAL.drop(['y'], axis=1)
y_val = VAL['y']
X_test = TEST.drop(['y'], axis=1)
y_test = TEST['y']

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

In [ ]:
lrs = [1e-2, 1e-3, 1e-4]
num_hidden_featuress = [64, 128, 256]
sigma_1s = [0.5, 1]
sigma_2s = [1e-6, 1e-3]
mixings = [0.2, 0.5, 0.8, 1] # mixing set to 1 make Gausiann Mixture a Normal Distribution

parameters = []
folders = []
os.makedirs("bayesian_neural_networks_models", exist_ok=True)
for lr in lrs:
    for num_hidden_features in num_hidden_featuress:
        for sigma_1 in sigma_1s:
            for mixing in mixings:
                for sigma_2 in sigma_2s:
                    # If mixing is equal to 1 than sigma_2 value has no meaning
                    if mixing == 1:
                        sigma_2 = 1e-6
                    
                    file_name = f'{lr}{num_hidden_features}{sigma_1}{sigma_2}{mixing}'
                    folder_name = os.path.join("bayesian_neural_networks_models", file_name)

                    if folder_name not in folders:
                        parameters.append((lr, num_hidden_features, sigma_1, sigma_2, mixing))
                        folders.append(folder_name)

                    if not os.path.exists(folder_name):
                        print(f'Training lr:{lr}, num_hidden_features:{num_hidden_features}, sigma_1:{sigma_1}, sigma_2:{sigma_2}, mixing:{mixing}')
                        set_seed(0)
    
                        model = BayesianMLP(
                            num_input_features=X_train.shape[1],
                            num_hidden_features=num_hidden_features, 
                            num_output_classes=len(y_train.unique()),
                            prior=TwoGaussianMixturePrior(sigma_1, sigma_2, mixing),
                        )
                        
                        loss_fn = ELBO(N=10)
                        optimizer = torch.optim.Adam(
                            model.parameters(), 
                            lr=lr, 
                        )
    
                        train_metrics, test_metrics = fit_elbo(
                            model=model,
                            train_dataset=train_dataset,
                            val_dataset=val_dataset,
                            loss_fn=loss_fn,
                            batch_size=BATCH_SIZE,
                            epochs=NO_EPOCHS,
                            optimizer=optimizer,
                        )

                        os.makedirs(folder_name, exist_ok=True)
    
                        torch.save(model, os.path.join(folder_name, MODEL_NAME))
    
                        elements = np.array([
                            train_metrics['loss'],
                            train_metrics['acc'],
                            train_metrics['step']
                        ]).transpose(1, 0)
    
                        train_df = pd.DataFrame(elements, columns=["loss", "acc", "step"])
                        train_df.to_csv(os.path.join(folder_name, TRAIN_CSV_NAME))
    
                        elements = np.array([
                            test_metrics['loss'],
                            test_metrics['acc'],
                            test_metrics['step']
                        ]).transpose(1, 0)
    
                        val_df = pd.DataFrame(elements, columns=["loss", "acc", "step"])
                        val_df.to_csv(os.path.join(folder_name, VAL_CSV_NAME))

In [ ]:
df_params = pd.DataFrame(parameters, columns=["lr", "num_hidden_features", "sigma_1", "sigma_2", "mixing"])
df_params.head()

In [ ]:
all_results = [get_results(f) for f in folders]
df_results = pd.DataFrame(all_results, columns=["train_loss", "train_acc", "val_loss", "val_acc"])
df_results.head()

In [ ]:
df = pd.merge(df_params, df_results, left_index=True, right_index=True)
df['folder'] = folders
df.head()

In [ ]:
pd.set_option('display.max_rows', None)
df_sorted = df.sort_values('val_loss')
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

## Tests - analiza wyników

In [ ]:
df[['lr', 'train_loss', 'train_acc', 'val_loss', 'val_acc']].groupby("lr").mean()

In [ ]:
df[['num_hidden_features', 'train_loss', 'train_acc', 'val_loss', 'val_acc']].groupby("num_hidden_features").mean()

In [ ]:
df[['sigma_1', 'train_loss', 'train_acc', 'val_loss', 'val_acc']].groupby("sigma_1").mean()

In [ ]:
df[df['mixing'] != 1][['sigma_2', 'train_loss', 'train_acc', 'val_loss', 'val_acc']].groupby("sigma_2").mean()

In [ ]:
df[['mixing', 'train_loss', 'train_acc', 'val_loss', 'val_acc']].groupby("mixing").mean()

In [ ]:
# We choose the best model and analyse it
best_model = get_model(df_sorted.iloc[0]['folder'])

In [ ]:
visualise_results(df_sorted.iloc[0]['folder'])

In [ ]:
calculate_metrics(best_model, X_test, y_test)

In [ ]:
num_samplings = 10
analyzer = Analyzer(best_model, test_dataset, num_samplings)

In [ ]:
print("Top high confidence correct predictions")
visualize_samples(
    *analyzer.get_top_k_high_confidence_correct(5),
    num_cols=5
)

In [ ]:
print("Top low confidence correct predictions")
visualize_samples(
    *analyzer.get_top_k_low_confidence_correct(5),
    num_cols=5
)

In [ ]:
print("Top low confidence wrong predictions")
visualize_samples(
    *analyzer.get_top_k_low_confidence_mistakes(5),
    num_cols=5
)

In [ ]:
print("Top high confidence wrong predictions")
visualize_samples(
    *analyzer.get_top_k_high_confidence_mistakes(5),
    num_cols=5
)

In [ ]:
visualize_weights(best_model.layer_1.weights_mean, "layer input -> hidden")
visualize_weights(best_model.layer_1.weights_std, "layer hidden -> output (std)")
print("Histogram of weights for layer 1")

In [ ]:
visualize_weights(best_model.layer_1.biases_mean, "layer input -> hidden")
visualize_weights(best_model.layer_1.biases_std, "layer hidden -> output (std)")
print("Histogram of biases for layer 1")

In [ ]:
visualize_weights(best_model.layer_2.weights_mean, "layer hidden -> output (mean)")
visualize_weights(best_model.layer_2.weights_std, "layer hidden -> output (std)")
print("Histogram of weights for layer 2")

In [ ]:
visualize_weights(best_model.layer_2.biases_mean, "layer input -> hidden")
visualize_weights(best_model.layer_2.biases_std, "layer hidden -> output (std)")
print("Histogram of biases for layer 2")

In [ ]:
# Credible intervals
plot_credible_intervals(X_test, y_test, 0)

In [ ]:
plot_credible_intervals(X_test, y_test, 1)

In [ ]:
plot_credible_intervals(X_test, y_test, 2)

In [ ]:
plot_credible_intervals(X_test, y_test, 3)

In [ ]:
plot_credible_intervals(X_test, y_test, 4)

In [ ]:
plot_credible_intervals(X_test, y_test, 5)

In [ ]:
plot_credible_intervals(X_test, y_test, 6)